In [1]:
import torch

# use a GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set default tensor type to float64
torch.set_default_dtype(torch.float64)

In [2]:
from botorch.test_functions.synthetic import Ackley
from botorch.utils.sampling import draw_sobol_samples

In [3]:
dim=4
ackley_function = Ackley(dim=dim)
scale_factor = -1
def objective_function(X):
    return ackley_function(2*X-1)/scale_factor
global_optimum_value = 0

In [4]:
bounds = torch.stack([torch.zeros(dim), torch.ones(dim)])
seed = 0
torch.manual_seed(seed)
init_x = draw_sobol_samples(bounds=bounds, n=1, q=2*(dim+1)).squeeze(0)

In [5]:
train_X = init_x
exact_Y = objective_function(init_x)

In [6]:
NOISE_SE = 0.05

In [7]:
train_Y = exact_Y + NOISE_SE * torch.randn_like(exact_Y)

In [8]:
from botorch.models import SingleTaskGP
from botorch.models.transforms.outcome import Standardize
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.fit import fit_gpytorch_model

In [10]:
train_Yvar = torch.full_like(train_Y, NOISE_SE**2)

if train_X.ndim == 1:
    train_x = train_X.unsqueeze(dim=-1)

if train_Y.ndim == 1:
    train_Y = train_Y.unsqueeze(dim=-1)

# Outcome transform
outcome_transform = Standardize(m=train_Y.shape[-1])

model = SingleTaskGP(train_X=train_X, train_Y=train_Y, train_Yvar=train_Yvar, outcome_transform=outcome_transform)

mll = ExactMarginalLogLikelihood(model.likelihood, model)

fit_gpytorch_model(mll)

ExactMarginalLogLikelihood(
  (likelihood): FixedNoiseGaussianLikelihood(
    (noise_covar): FixedGaussianNoise()
  )
  (model): SingleTaskGP(
    (likelihood): FixedNoiseGaussianLikelihood(
      (noise_covar): FixedGaussianNoise()
    )
    (mean_module): ConstantMean()
    (covar_module): ScaleKernel(
      (base_kernel): MaternKernel(
        (lengthscale_prior): GammaPrior()
        (raw_lengthscale_constraint): Positive()
      )
      (outputscale_prior): GammaPrior()
      (raw_outputscale_constraint): Positive()
    )
    (outcome_transform): Standardize()
  )
)

In [11]:
from botorch.acquisition.analytic import NoisyExpectedImprovement
from botorch.optim import optimize_acqf

NEI = NoisyExpectedImprovement(model, train_X)
new_point, new_point_NEI = optimize_acqf(
        acq_function=NEI,
        bounds=bounds,
        q=1,
        num_restarts=10*dim,
        raw_samples=200*dim,
        options={'method': 'L-BFGS-B'},
)

RuntimeError: Error(s) in loading state_dict for SingleTaskGP:
	Unexpected key(s) in state_dict: "outcome_transform.means", "outcome_transform.stdvs", "outcome_transform._stdvs_sq", "outcome_transform._is_trained". 